In [1]:
import time
import instaloader
import pandas as pd
import os
from datetime import datetime, timedelta
import locale
import shutil

In [2]:
import logging

In [3]:
logFilePath = "../logConcorrencia.txt"

In [4]:
filePath = "../AnaliseConcorrentes.xlsx"
fileCopyPath = "../AnaliseConcorrentesCopy.xlsx"

In [5]:
# Verificar se o arquivo original existe
if os.path.exists(filePath):
    # Criar uma cópia do arquivo com o novo nome
    shutil.copy2(filePath, fileCopyPath)
    print(f'O arquivo "{filePath}" foi copiado como "{fileCopyPath}".')
else:
    print(f'O arquivo "{filePath}" não foi encontrado.')

O arquivo "../AnaliseConcorrentes.xlsx" foi copiado como "../AnaliseConcorrentesCopy.xlsx".


In [6]:
# Configuração básica do log
logging.basicConfig(filename=logFilePath, 
                    level=logging.ERROR, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [7]:
# Função para capturar e registrar erros
def log_error(error_type, error_description):
    logging.error(f'{error_type}: {error_description}')

In [ ]:
# inicializa o instaloader
loader = instaloader.Instaloader()

# Lista de usernames do Instagram
usernames = ['usuario_monitorado1', 'usuario_monitorado2', 'usuario_monitorado3']

In [17]:
# Define a localidade para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

# Obtém o mês e ano atual em português
mes_ano_atual = datetime.now().strftime("%B/%Y") 

# Tenta carregar o arquivo Excel existente, se existir
if os.path.exists(filePath):
    df = pd.read_excel(filePath, index_col=0)
    df.reset_index(inplace=True)
else:
    # Cria um novo DataFrame se o arquivo não existir
    df = pd.DataFrame(columns=["username", mes_ano_atual, "CRESCIMENTO", "CRESCIMENTO (%)"])

# Lista para armazenar os novos dados
novos_dados = []

# Itera sobre cada usuário
for username in usernames:
    try:
        # pega dados do perfil
        profile = instaloader.Profile.from_username(loader.context, username)
        print(f"{username} - Posts: {profile.mediacount}, Followers: {profile.followers}, Following: {profile.followees}")

        seguidores_atual = profile.followers

        # verifica se existe algum username 
        if username in df['username'].values:
            # Atualiza os dados existentes
            index = df[df['username'] == username].index[0]


            # Verifica se a coluna do mês atual já existe, se não, cria com valores '-'
            if mes_ano_atual not in df.columns:
                df[mes_ano_atual] = '-'

            # Obtém o número de seguidores do mês anterior
            seguidores_anterior = df.loc[index, df.columns[-4]]

            print("Seguidores antigos: {} ".format(seguidores_anterior))
            print("Seguidores novos: {}".format(seguidores_atual))
            
            # Calcula o crescimento e crescimento percentual
            crescimento = seguidores_atual - seguidores_anterior
            crescimento_percentual = (crescimento / seguidores_anterior) if seguidores_anterior != 0 else 0

            # Atualiza as colunas com os novos dados
            df.at[index, mes_ano_atual] = seguidores_atual
            df.at[index, "CRESCIMENTO"] = crescimento
            df.at[index, "CRESCIMENTO (%)"] = crescimento_percentual
        else:
            # Adiciona uma nova linha para o novo usuário
            novos_dados.append({
                "username": username,
                mes_ano_atual: seguidores_atual,
                "CRESCIMENTO": "-",
                "CRESCIMENTO (%)": "-"
            })
        
        # Pausa para evitar atingir limites de requisições
        time.sleep(10)  # Espera de 10 segundos entre requisições

    except instaloader.exceptions.ConnectionException as co:
        log_error(type(co).__name__, str(co))
        print(f"Erro de conexão com o perfil {username}. Esperando para tentar novamente.")
        time.sleep(300)  # Espera de 5 minutos antes de tentar novamente

    except instaloader.exceptions.LoginRequiredException as lr:
        log_error(type(lr).__name__, str(lr))
        print(f"Login Required. Último perfil {username}. Esperando para tentar novamente.")
        time.sleep(300)

if novos_dados:
    df_novos_dados = pd.DataFrame(novos_dados)
    df = pd.concat([df, df_novos_dados], ignore_index=True)

maisgoias - Posts: 139428, Followers: 1666668, Following: 704
Seguidores antigos: 1642093 
Seguidores novos: 1666668
goianiamilgraus - Posts: 15291, Followers: 583048, Following: 1465
Seguidores antigos: 582582 
Seguidores novos: 583048
enquantoissoemgoias - Posts: 26232, Followers: 839624, Following: 400
Seguidores antigos: 834438 
Seguidores novos: 839624
tvanhanguerag1go - Posts: 11217, Followers: 304048, Following: 71
Seguidores antigos: 294483 
Seguidores novos: 304048
maisgoiasoficial - Posts: 5112, Followers: 324941, Following: 17
Seguidores antigos: 323269 
Seguidores novos: 324941
portal6noticias - Posts: 39497, Followers: 281440, Following: 71
Seguidores antigos: 276186 
Seguidores novos: 281440
jornal_opopular - Posts: 52797, Followers: 562505, Following: 38
Seguidores antigos: 559809 
Seguidores novos: 562505
ohoje - Posts: 45757, Followers: 80214, Following: 10
Seguidores antigos: 79215 
Seguidores novos: 80214
guiacurtamais - Posts: 27643, Followers: 311985, Following: 19

In [19]:
# cria coluna com o prox mês com valores '-'
mes_ano_seguinte = (datetime.now().replace(day=28) + timedelta(days=4)).strftime("%B/%Y")
df[mes_ano_seguinte] = '-'

In [21]:
# coloca CRESCIMENTO e CRESCIMENTO % no final
colunas_finais = [col for col in df.columns if col not in ["CRESCIMENTO", "CRESCIMENTO (%)"]]
df = df[colunas_finais + ["CRESCIMENTO", "CRESCIMENTO (%)"]]

In [23]:
# Salva os dados no arquivo Excel
df.to_excel(filePath, index=False)

In [ ]:
df